# Connecter le drive et le colab

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=False)

In [ ]:
import os
os.listdir()
#cwd = os.getcwd()
#print(cwd)
#!ls gdrive/MyDrive
#os.listdir()
path = "/content/gdrive/MyDrive"
os.chdir(path)
cwd = os.getcwd()


**Import packages**

In [1]:
import os
import glob
import pandas as pd
import numpy as np
import cv2
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
import re
tqdm_notebook.pandas()
import tensorflow as tf
#from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn import metrics
import itertools
import pickle
from joblib import dump, load

# Identification of product classes

 ***Create xtrain , xtest and ytrain DataFrames from csv files***

In [ ]:
df_xtrain = pd.read_csv('./data/texts/X_train_update.csv') 

In [ ]:
df_xtest = pd.read_csv('./data/texts/X_test_update.csv') 

In [ ]:
df_ytrain = pd.read_csv('./data/texts/Y_train_CVw08PX.csv') 

In [ ]:
#print("df_xtrain shape: ", df_xtrain.shape)
#print("df_xtest shape: ", df_xtest.shape)
#print("df_ytrain shape: ", df_ytrain.shape)

In [ ]:
df_xtrain.head()

In [ ]:
df_xtest.head()

In [ ]:
df_ytrain.head()

In [ ]:
# Nan values
null_counts = df_xtrain.isnull().sum().sort_values(ascending=False)
print('df_xtrain: \n', null_counts[null_counts > 0])

null_counts = df_xtest.isnull().sum().sort_values(ascending=False)
print(' \n df_xtest: \n',null_counts[null_counts > 0])

null_counts = df_ytrain.isnull().sum().sort_values(ascending=False)
print(' \ndf_ytrain: \n',null_counts[null_counts > 0])

In [ ]:
display(df_ytrain["prdtypecode"].unique())

unique, counts = np.unique(df_ytrain["prdtypecode"], return_counts=True)
dict(zip(unique, counts))

In [ ]:
df_ytrain["prdtypecode"].value_counts(normalize=True).mul(100).round(2).astype(str) + '%'

 ***Concatenate  df_xtrain and df_ytrain(to get product code type(Classes))***

In [ ]:
# Sort des deux DF avant de faire la concaténation
df_xtrain = df_xtrain.sort_values(by = 'Unnamed: 0', ascending = True)
df_ytrain = df_ytrain.sort_values(by = 'Unnamed: 0', ascending = True)

In [ ]:
# Comparaison des deux colonnes 'Unnamed: 0' des deux DF df_xtrain et  df_ytrain
df_xtrain['Unnamed: 0'].isin(df_ytrain['Unnamed: 0']).value_counts()

In [ ]:
# Concaténation des deux DF - Train
df_combined = pd.concat([df_xtrain, df_ytrain['prdtypecode'] ], axis = 1)
df_combined.drop('Unnamed: 0', axis =1 , inplace = True)

In [ ]:
df_combined

In [ ]:
df_combined.info()

In [ ]:
df_combined.shape

***Get image name from product and image Id***

In [ ]:
#path_im = "./data/images/all/image_train/"  # Train folder containing org image  500 x 500 
#path_im = "./data/images/all/image_train_resized/"  # Train folder containing resized  256 x 256 
#im_prfix = "image_"
im_prfix = "resized_image_"

#df_combined["imagePath"] =  path_im  + im_prfix  + df_combined.imageid.astype(str)+ "_product_" + df_combined.productid.astype(str) + ".jpg"
df_combined["imagePath"] =  im_prfix  + df_combined.imageid.astype(str)+ "_product_" + df_combined.productid.astype(str) + ".jpg"

In [ ]:
df_combined

# Dataframe saving to pickle and CSV files

In [ ]:
from datetime import date
import xlwt
import openpyxl
pathSave = './saves/df_save/'

today = date.today()
version = '_V0'

# To pickle file
df_combined.to_pickle(f"{pathSave}/df_train_combined_19102021.pkl")

# To CSV file
df_combined.to_csv(f"{pathSave}/df_train_combined_19102021{today}{version}.csv")

# Iteration #1 - ResNet50(Residual Networks)

***Read DataFrame from saved pickele file - image Train set***

In [3]:
df_train_im = pd.read_pickle("./saves/df_save/df_train_combined_19102021.pkl")

In [4]:
df_train_im.head()

,designation,description,productid,imageid,prdtypecode,imagePath
0,Olivia: Personalisiertes Notizbuch / 150 Seite...,NaN,3804725264,1263597046,10,resized_image_1263597046_product_3804725264.jpg
1,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,NaN,436067568,1008141237,2280,resized_image_1008141237_product_436067568.jpg
2,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,PILOT STYLE Touch Pen de marque Speedlink est ...,201115110,938777978,50,resized_image_938777978_product_201115110.jpg
3,Peluche Donald - Europe - Disneyland 2000 (Mar...,NaN,50418756,457047496,1280,resized_image_457047496_product_50418756.jpg
4,La Guerre Des Tuques,Luc a des id&eacute;es de grandeur. Il veut or...,278535884,1077757786,2705,resized_image_1077757786_product_278535884.jpg


In [5]:
display(df_train_im.shape)

(84916, 6)

In [6]:
#df_train_im = df_train_im.head(df_train_im.shape[0]//2)

***Change of product codes from 27 classes to 0 to 26***

In [7]:
df_train_im.replace({'prdtypecode':{10:0,
                                    40:1,
                                    50:2,
                                    60:3,
                                    1140:4,
                                    1160:5,
                                    1180:6,
                                    1280:7,
                                    1281:8,
                                    1300:9                                    
                                    }}, inplace = True)

df_train_im.replace({'prdtypecode':{1301:10,
                                    1302:11,
                                    1320:12,
                                    1560:13,
                                    1920:14,
                                    1940:15,
                                    2060:16,
                                    2220:17,
                                    2280:18,
                                    2403:19,
                                    2462:20,
                                    2522:21,
                                    2582:22,
                                    2583:23,
                                    2585:24,
                                    2705:25,
                                    2905:26
                                    }}, inplace = True)

In [8]:
display(df_train_im["prdtypecode"].unique())

array([ 0, 18,  2,  7, 25, 21, 22, 13,  8, 14, 19,  4, 23,  6,  9, 20,  5,
       16,  1,  3, 12, 11, 17, 26, 24, 15, 10], dtype=int64)

 ***Split Train Set***

In [9]:
# Split Train set 
#X_train_im, X_test_im = train_test_split(df_train_im, train_size=0.8, random_state=1234)

In [10]:
#display(X_train_im.shape)
#display(X_test_im.shape)

***Training the model***

***Data Generator***

In [11]:
df_train_im["prdtypecode"] = df_train_im["prdtypecode"].astype(str)

In [13]:
# Set column as str type - for data generation
#X_train_im["prdtypecode"] = X_train_im["prdtypecode"].astype(str)
#X_test_im["prdtypecode"] = X_test_im["prdtypecode"].astype(str)


In [12]:
unique, counts = np.unique(df_train_im["prdtypecode"], return_counts=True)
dict(zip(unique, counts))

{'0': 3116,
 '1': 2508,
 '10': 807,
 '11': 2491,
 '12': 3241,
 '13': 5073,
 '14': 4303,
 '15': 803,
 '16': 4993,
 '17': 824,
 '18': 4760,
 '19': 4774,
 '2': 1681,
 '20': 1421,
 '21': 4989,
 '22': 2589,
 '23': 10209,
 '24': 2496,
 '25': 2761,
 '26': 872,
 '3': 832,
 '4': 2671,
 '5': 3953,
 '6': 764,
 '7': 4870,
 '8': 2070,
 '9': 5045}

In [13]:
%%time
#Générateur de données
img_gen = ImageDataGenerator(rescale = 1/255, 
                             validation_split = 0.2)

Wall time: 0 ns


In [15]:
img_rows = 128
img_cols = 128
batch_size = 32
epochs = 10
images_dir_train = './data/images/all/image_train_resized/'

In [16]:
%%time

#Itérateur 
train_generator = img_gen.flow_from_dataframe(dataframe = df_train_im,
                                              directory = images_dir_train,
                                              as_ext = True,
                                              x_col = 'imagePath',
                                              y_col = 'prdtypecode',  #target data                                            
                                              class_mode = 'sparse',                                             
                                              target_size = (img_rows , img_cols), # default 256 x 256                                             
                                              batch_size = batch_size,                                             
                                              shuffle = True, # défault
                                              subset = 'training'
                                             )

Found 67933 validated image filenames belonging to 27 classes.
Wall time: 7min 4s


In [17]:
%%time
# validation_split the validation batches ca be retrieved by specifying the subset as validation.
valid_generator = img_gen.flow_from_dataframe(dataframe = df_train_im, 
                                              directory = images_dir_train,
                                              as_ext = True,
                                              x_col = 'imagePath',
                                              y_col = 'prdtypecode',  #target data                                            
                                              class_mode = 'sparse',                                        
                                              target_size = (img_rows , img_cols), # default 256 x 256                                         
                                              batch_size = batch_size, 
                                              shuffle = False,
                                              subset  = 'validation'
                                             )

Found 16983 validated image filenames belonging to 27 classes.
Wall time: 11.2 s


 ***ResNet50 Implementation and Compilation***

In [18]:
%%time
# Freez des chouche pré_entrainées d'Extraction de Features
def make_model():
    base_model = ResNet50(
        include_top = False,
        weights ='imagenet',
        input_shape = None, #as bmark (img_rows,img_rows,3)
        pooling ='avg'
    )
    for layer in base_model.layers:
        layer.trainable = False

    predictions = Dense(27, activation='sigmoid')(base_model.output)  # sigmoid à re vérifier !!!!!!!!!!!!
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    return model

model = make_model()
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

***Callbacks***

In [19]:
%%time
path = './saves/ResNet50/'
file_name = 'checkpoint_ResNet50_06112021-10Epochs.h5'

#Sauvegarder les meilleurs poids du modèle au cours de l'entraînement :
checkpoint = ModelCheckpoint(filepath = path + file_name , 
                             monitor ='val_accuracy',
                             verbose = 1,
                             save_best_only = True, # (pour que le meilleur modèle ne soit pas écrasé)
                             save_weights_only = False,
                             mode ='max', #(permet de préciser si la métrique doit croître ou décroitre : ici on choisit 'min'
                                           #car la métrique est une perte à minimiser)
                             save_freq='epoch')
#Arrêter l'entraînement si le modèle n'évolue plus (très pratique pour ne pas gérer le
#nombre d'epoch) :
early = EarlyStopping(monitor='val_loss',
                      min_delta = 0,
                      patience = 3, #(nombre d'epochs à attendre avant d'arrêter l'entraînement
                      restore_best_weights = True,
                      verbose = 1,
                      mode = 'min')



Wall time: 0 ns


 ***Fit - train_generator***

In [20]:
step_size_Train = train_generator.n//train_generator.batch_size
step_size_Valid = valid_generator.n//valid_generator.batch_size
print('step_size_Train : ' , step_size_Train)
print('step_size_Valid  : ' , step_size_Valid)

step_size_Train :  2122
step_size_Valid  :  530


In [21]:
%%time
# Fit Train generator
history = model.fit(train_generator,                    
                    epochs = epochs,                   
                    steps_per_epoch = step_size_Train,
                    validation_data = valid_generator,
                    validation_steps = step_size_Valid,
                    callbacks=[checkpoint,early]
                    
                    )

Epoch 1/10
2122/2122 [==============================] - ETA: 0s - loss: 2.8695 - accuracy: 0.1708
Epoch 00001: val_accuracy improved from -inf to 0.20018, saving model to ./saves/ResNet50\checkpoint_ResNet50_06112021-10Epochs.h5
2122/2122 [==============================] - 6154s 3s/step - loss: 2.8695 - accuracy: 0.1708 - val_loss: 2.7516 - val_accuracy: 0.2002
Epoch 2/10
2122/2122 [==============================] - ETA: 0s - loss: 2.7065 - accuracy: 0.2209
Epoch 00002: val_accuracy improved from 0.20018 to 0.22983, saving model to ./saves/ResNet50\checkpoint_ResNet50_06112021-10Epochs.h5
2122/2122 [==============================] - 5769s 3s/step - loss: 2.7065 - accuracy: 0.2209 - val_loss: 2.6721 - val_accuracy: 0.2298
Epoch 3/10
2122/2122 [==============================] - ETA: 0s - loss: 2.6438 - accuracy: 0.2384
Epoch 00003: val_accuracy improved from 0.22983 to 0.24858, saving model to ./saves/ResNet50\checkpoint_ResNet50_06112021-10Epochs.h5
2122/2122 [==========================

 ***Prediction - Valid_generator***

In [22]:
%%time
#Prediction - Test set
y_pred_proba = model.predict(valid_generator)

# l'argmax pour obtenir les classes prédites
y_pred_class = np.argmax(y_pred_proba,axis = 1).astype(int)

# To get classes from test generator
y_true = valid_generator.classes

Wall time: 19min 36s


 ***Evaluation - Valid_generator***

In [23]:
%%time
#Get the accuracy score
test_score = model.evaluate(valid_generator)

print("[INFO] accuracy: {:.2f}%".format(test_score[1] * 100)) 
print("[INFO] Loss: ",test_score[0])

531/531 [==============================] - 1322s 2s/step - loss: 2.5017 - accuracy: 0.2912
[INFO] accuracy: 29.12%
[INFO] Loss:  2.5016567707061768
Wall time: 22min 5s


In [24]:
%%time
print(metrics.classification_report(y_true, y_pred_class))

              precision    recall  f1-score   support

           0       0.32      0.23      0.27       621
           1       0.40      0.22      0.28       483
           2       0.29      0.01      0.02       154
           3       0.39      0.03      0.05       508
           4       0.21      0.07      0.11       656
           5       0.34      0.07      0.11      1038
           6       0.49      0.41      0.45       848
           7       0.00      0.00      0.00       152
           8       0.18      0.29      0.22      1021
           9       0.00      0.00      0.00       168
          10       0.26      0.56      0.36       956
          11       0.33      0.39      0.36       939
          12       0.25      0.03      0.05       375
          13       0.37      0.08      0.14       290
          14       0.30      0.27      0.28       949
          15       0.10      0.00      0.00       508
          16       0.26      0.73      0.39      2061
          17       0.20    

C:\Users\O2P\anaconda3\envs\Mon_env_DataScienTest\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\O2P\anaconda3\envs\Mon_env_DataScienTest\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\O2P\anaconda3\envs\Mon_env_DataScienTest\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

In [25]:
unique, counts = np.unique(y_pred_class, return_counts=True)
dict(zip(unique, counts))

{0: 435,
 1: 259,
 2: 7,
 3: 33,
 4: 221,
 5: 205,
 6: 707,
 8: 1689,
 9: 3,
 10: 2053,
 11: 1094,
 12: 40,
 13: 65,
 14: 848,
 15: 10,
 16: 5682,
 17: 46,
 18: 789,
 19: 128,
 20: 7,
 21: 166,
 22: 608,
 23: 3,
 24: 615,
 25: 31,
 26: 1239}

 ***Save ResNet Model***

In [26]:
#Enregistrement du modèle 
model.save("./saves/ResNet50/ResNet50_All_train_data_06112021_10Epochs.hdf5")

#The SavedModel and HDF5 file contains:
#the model's configuration (topology)
#the model's weights
#the model's optimizer's state (if any)

 ***Load ResNet Model***

In [ ]:
#Rechargement du modèle :
model_saved = load_model('./saves/ResNet50/ResNet50_All_train_data_06112021_10Epochs.hdf5')
